In [46]:
import pandas as pd
import numpy as np
import math
import datetime

table=pd.read_csv("../Dati/orari/route.csv", delimiter=";")
time_table=pd.read_csv("../Dati/orari/route_time.csv", delimiter=";")
dft = pd.read_csv("../Dati/distanze/times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("../Dati/distanze/distances_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
fur1 = pd.read_csv("../Dati/turni_f/furgoncino1_routing.csv", delimiter=";")
fur2 = pd.read_csv("../Dati/turni_f/furgoncino2_routing.csv", delimiter=";")
fur1d = pd.read_csv("../Dati/turni_f/furgoncino1_KM_routing.csv", delimiter=";")
fur2d = pd.read_csv("../Dati/turni_f/furgoncino2_KM_routing.csv", delimiter=";")

def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)

def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

def getName(nome):
    city = dft[get_row(dft, nome).values].index[0]
    return city

In [47]:
def new_route(schedule, dft):
    times = []
    keys = []
    #row = dft[get_row(dft, 'Cremona, sesto 39').values]
    row = dft[get_row(dft, schedule[0]).values]
    key1 = ''
    key2 = ''
    impossibile = []
    
    for i in range(1,len(schedule)):
        key1 = row.index[0]
        try:
            paese=schedule[i]
        except:
            continue
        #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    
    steps_times1 = pd.Series(times , keys)
    times1 = times[:]
    return steps_times1

In [48]:
#first attempt to local search algorithm
def find_best_position(steps_times, to_insert, dft):
    times = []
    key2 = []
    for i in range(len(steps_times)):
        cities = steps_times.keys()[i].split('-')
        row1 = dft[get_row(dft, cities[0].strip()).values]
        #time from city 1 to current (to insert)
        f_c1_to_curr = row1.loc[:,get_column(dft, to_insert).values].values[0][0]
        row2 = dft[get_row(dft, to_insert).values]
        #time from current (to insert) to city 2
        f_curr_to_c2 = row2.loc[:,get_column(dft, cities[1].strip()).values].values[0][0]
        times.append(f_c1_to_curr + f_curr_to_c2)
    s3 = pd.Series(times, steps_times.keys())
    return s3

In [49]:
#returns a pd.Series: keys = city1 - city2, values = times from city1 to city2
def schedule_time(new_schedule, dft):
    times = []
    #row = dft[get_row(dft, 'Cremona, sesto 39').values]
    row = dft[get_row(dft, new_schedule[0]).values]
    keys = []
    key1 = ''
    key2 = ''
    for i in range(1,len(new_schedule)):
        key1 = row.index[0]
        paese=new_schedule[i]

        value=row.loc[:,get_column(dft, paese).values].values[0][0]
        times.append(round(value))
        
        row = dft[get_row(dft, paese).values]
        key2 = row.index[0]
        keys.append(key1 + ' - ' + key2)
    steps_times = pd.Series(times , keys)
    return steps_times

In [65]:
def timeDistance(start, end, dft):
    start_city_row = dft[get_row(dft, start).values]
    return start_city_row.loc[:,get_column(dft, end).values].values[0][0]

In [51]:
#Data una lista ottenuta da route restituisce la lista dei nomi delle città
def cities_list(serie):
    cities = []
    keys = serie.keys()
    for i in range(0, len(serie)):
        cities = cities + [keys[i].split('-')[0].strip()]
    if len(keys[i].split('-')) > 2: 
        return cities + [keys[i].split('-')[len(keys[i].split('-'))-1].strip()]
    else:
        return cities + [keys[i].split('-')[1].strip()]

In [52]:
def best_index(lista, schedule, to_insert, dft):
    idx = find_best_position(schedule, to_insert, dft).idxmin()
    a = idx.split("-")[0].strip()    
    try:
        return lista.index(a)+1
    except:
        #problema nome diverso
        if(a == 'Casale Cremasco'):
            a = 'Casale Cremasco-Vidolasco, Roma, 56'
            return lista.index(a)+1
        elif(city == 'gadesco'):
            a = 'Gadesco-Pieve Delmona, Giovanni Lonati 9'
            return lista.index(a)+1

In [66]:
def definitive_local_search(fur, n, dft):
    #Schedulazione iniziale, e sequenza dei nomi iniziali, una per ciclare e una copia per effettuare pop e insert
    pre_schedule = new_route(fur, dft)
    lista = [getName(x) for x in fur]
    sequence = lista[:]
    #Costo iniziale del percorso greedy
    if(n==1):
         pre_cost = sum(pre_schedule.values) + timeDistance('Cremona, Sesto 39', lista[0], dft) + timeDistance(lista[-1], 'Ostiano, Mazzini, 1', dft)
    else:
         pre_cost = sum(pre_schedule.values) + timeDistance('Cremona, Sesto 39', lista[0], dft) + timeDistance(lista[-1], 'San Giovanni in Croce, Feudatari, 25 A', dft)
    new_cost = 0
    local_best_cost = math.inf
    local_best_schedule = []

    #Eseguo finchè ho dei miglioramenti e quindi i nuovi costi sono migliori del migliore al passo prima
    while(True):
        #Prendo ogni città in lista, la inserisco nella posizione migliore, e tra tutte le città prendo quella che da lo
        #spostamento più vantaggioso.
        #Se questo è anche migliore del risultato precedente utilizzo la nuova sequenza come passo del ciclo successivo
        for i in range(len(lista)):
            #Copia della lista su cui lavorare
            sequence = lista[:]
            to_insert = lista[i]
            sequence.remove(to_insert)
            curr_schedule = new_route(sequence, dft)
            index = best_index(sequence, curr_schedule, to_insert, dfd)
            sequence.insert(index, to_insert)
            curr_schedule = new_route(sequence, dft)
            if(n==1):
                new_cost = sum(curr_schedule.values) + timeDistance('Cremona, Sesto 39', sequence[0], dft) + timeDistance(sequence[-1], 'Ostiano, Mazzini, 1', dft)
            else:
                new_cost = sum(curr_schedule.values) + timeDistance('Cremona, Sesto 39', sequence[0], dft) + timeDistance(sequence[-1], 'San Giovanni in Croce, Feudatari, 25 A', dft)
            if(new_cost<local_best_cost):
                local_best_cost = new_cost
                local_best_schedule = curr_schedule[:]
        if(local_best_cost<pre_cost):
            pre_cost = local_best_cost
            pre_schedule = local_best_schedule[:]
            lista = cities_list(pre_schedule)
        else:
            break
    return cities_list(pre_schedule), pre_cost

In [55]:
#TIME LOCAL SEARCH
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur1[day].dropna(), 1, dft)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('Ostiano, Mazzini, 1')
    to_write[day] = pd.Series(sch)

In [13]:
to_write.to_csv("../Dati/turni_f/ls1_shifts.csv", index=False, sep=";")

In [ ]:
week= ['martedi', 'mercoledi', 'giovedi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur1[day].dropna(), 2, dft)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('San Giovanni in Croce, Feudatari, 25 A')
    print(sch)
    print("\n")
    to_write[day] = pd.Series(sch)
to_write.to_csv("../Dati/turni_f/ls2_shifts.csv", index=False, sep=";")

In [67]:
#COMPUTE KM LOCAL SEARCH
week= ['lunedi', 'martedi', 'mercoledi', 'giovedi', 'venerdi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search(fur1[day].dropna(), 1, dfd)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('Ostiano, Mazzini, 1')
    to_write[day] = pd.Series(sch)
to_write.to_csv("../Dati/turni_f/ls1_km_shifts.csv", index=False, sep=";")

,lunedi,martedi,mercoledi,giovedi,venerdi
0,"Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39","Cremona, Sesto 39"
1,"Sesto ed Uniti, Roma, 4","Cremona, Ugolani Dati, 4","Castelverde, Ubaldo Ferrari, 15","Cremona, Sesto 39","Crema, Civerchi, 9"
2,"Grumello Cremonese ed Uniti, Roma, 2","Persico Dosimo, Albert ,7","Annicco, Giuseppe Garibaldi, 6","Cremona, Ugolani Dati, 4","Capergnanica, XI febbraio, 7"
3,"Pizzighettone, Giuseppe Garibaldi, 18","Ostiano, Mazzini, 1","Castelleone, Roma, 67","Persico Dosimo, Albert ,7","Bagnolo Cremasco, Roma, 3"
4,"Soresina, Giacomo Matteotti, 6","San Giovanni in Croce, Feudatari, 25 A","Madignano, Portici, 4","Grontardo, Roma, 16","Vaiano Cremasco, Gloriosi Caduti, 3"
5,"Trigolo, Roma, 26","Casalmaggiore, Guglielmo Marconi, 8","Bagnolo Cremasco, Roma, 3","Vescovato, Roma, 44","Monte Cremasco, Roma, 12"
6,"Castelleone, Roma, 67","Scandolara Ravara, Italia","Palazzo Pignano, Marco Polo, 31","Ostiano, Mazzini, 1","Dovera, Oldrini 2"
7,"Montodine, XXV Aprile, 10","Casalpusterlengo, Pusterla 7","Pandino, Castello, 15","Isola Dovarese, Giacomo Matteotti, 1","Pandino, Castello, 15"
8,"Ripalta Cremasca, Roma, 5","Vailate, Dante Alighieri, 7","Rivolta d'Adda, Stefano Renzi, 5","Piadena, Platina, 40","Agnadello, Marconi, 6"
9,"Capralba, Piave, 2","Crema, Civerchi, 9","Spino d'Adda, Circonvallazione, 13","San Giovanni in Croce, Feudatari, 25 A","Rivolta d'Adda, Stefano Renzi, 5"


In [60]:
to_write.to_csv("../Dati/turni_f/ls1_km_shifts.csv", index=False, sep=";")

In [61]:
week= ['martedi', 'mercoledi', 'giovedi']
data={}
to_write = pd.DataFrame(data)
for day in week:
    sch, cst = definitive_local_search_km(fur1[day].dropna(), 2, dfd)
    sch.insert(0, 'Cremona, Sesto 39')
    sch.append('San Giovanni in Croce, Feudatari, 25 A')
    to_write[day] = pd.Series(sch)
to_write.to_csv("../Dati/turni_f/ls2_km_shifts.csv", index=False, sep=";")